### Analysis of Movie ratings dataset “Movie Lens Dataset”

Import required libraries and intialize spark context

In [9]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("MovieRating").getOrCreate()
sc=spark.sparkContext


<p color>Q a) How many lines does the ratings file contain?<p>

In [10]:
ratings_file=sc.textFile(r'/content/sample_data/ratings.dat')
number_of_lines=ratings_file.count()
print(f'The ratings.dat file contains {number_of_lines} lines')

The ratings.dat file contains 1000209 lines


Q b) How many times the rating “1” has been given?

In [11]:
splitted_ratings_data = ratings_file.map(lambda line1: line1.split('::'))

In [12]:
count_rating_1 = splitted_ratings_data.map(lambda line: line[2]).filter(lambda col: col == '1').count()
print(f'Rating 1 was given {count_rating_1} times.')

Rating 1 was given 56174 times.


Q c) Which are the 10 most popular movies?

In [13]:
movid_rating_rdd = splitted_ratings_data.map(lambda item : (item[1], int(item[2])))
movid_total_ratings_rdd = movid_rating_rdd.groupByKey().mapValues(lambda values: sum(values))
movid_top10_rating_rdd = sc.parallelize(movid_total_ratings_rdd.sortBy(lambda x: x[1], ascending=False).take(10))

Load moving.dat to get movie names

In [19]:
movies_file=sc.textFile(r'/content/sample_data/movies.dat')
splitted_movies_data = movies_file.map(lambda line1: line1.split('::'))


Print final result

In [21]:
final_result = movid_top10_rating_rdd.join(splitted_movies_data).map(lambda x: (x[0], x[1][0], x[1][1])).collect()


In [22]:
print(f'Following are top 10 most popular moviews based on consolidated user ratings:')
for itr, item in enumerate(final_result, start=1):
  print(f'#{itr} {item[2]}, {item[1]}')

Following are top 10 most popular moviews based on consolidated user ratings:
#1 Star Wars: Episode V - The Empire Strikes Back (1980), 12836
#2 Star Wars: Episode VI - Return of the Jedi (1983), 11598
#3 Saving Private Ryan (1998), 11507
#4 Raiders of the Lost Ark (1981), 11257
#5 Sixth Sense, The (1999), 10835
#6 American Beauty (1999), 14800
#7 Star Wars: Episode IV - A New Hope (1977), 13321
#8 Silence of the Lambs, The (1991), 11219
#9 Matrix, The (1999), 11178
#10 Terminator 2: Judgment Day (1991), 10751
